<a href="https://colab.research.google.com/github/nahiim/colab/blob/main/07_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the CNN architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolutional Layer 1: (Input: 1 channel, Output: 32 channels)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # Max Pooling Layer 1: (Downsamples the feature maps)
        self.pool1 = nn.MaxPool2d(2, 2)

        # Convolutional Layer 2: (Input: 32 channels, Output: 64 channels)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Max Pooling Layer 2
        self.pool2 = nn.MaxPool2d(2, 2)

        # Fully Connected Layer: (Input: Flattened 64 channels from the last conv layer)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Flatten 7x7x64 to a vector
        self.fc2 = nn.Linear(128, 10)  # Output layer: 10 classes for MNIST digits

    def forward(self, x):
        # Pass the input through conv1, pool1, conv2, pool2, and then fully connected layers
        x = self.pool1(torch.relu(self.conv1(x)))  # Convolution + ReLU + Pooling
        x = self.pool2(torch.relu(self.conv2(x)))  # Convolution + ReLU + Pooling
        x = x.view(-1, 64 * 7 * 7)  # Flatten the output to 1D vector
        x = torch.relu(self.fc1(x))  # Fully connected layer with ReLU
        x = self.fc2(x)  # Output layer
        return x

# Load the MNIST dataset and apply the necessary transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()  # Loss function for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer (Adam)

# Training Loop
for epoch in range(5):
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear previous gradients

        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss

        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        total_loss += loss.item()  # Accumulate loss for the epoch

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

# Testing the model
correct = 0
total = 0
with torch.no_grad():  # No gradients needed for testing
    for images, labels in test_loader:
        outputs = model(images)  # Get model predictions
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Epoch 1 | Loss: 147.0945
Epoch 2 | Loss: 43.1002
Epoch 3 | Loss: 28.9569
Epoch 4 | Loss: 21.7810
Epoch 5 | Loss: 16.3824
Accuracy on test set: 99.15%
